This project focuses on modeling and implementing two FX Swaption pricing models for USDEUR pair and compare them. This can be considered as a case of Exotic Options (as FX Swaptions can be considered a type of exotic option or a derivative on a currency swap) or generally working with derivative instruments.

The project is structrured in the following 5 parts:

1. Data Collection (FX, Rates)
2. Calibration/Interpolation (Yield Curves, Volatility Smile)
3. Benchmark Model (Black Model)
4. Alternative Model (Merton Jump-Diffusion via Monte Carlo)
5. Comparison


**Part 1: Imports and Data Collection**

This code uses yfinance for FX data and fredapi for USD rates, as they are reliable and free.

The goal is to gather the three key inputs needed for any FX option or cross-currency derivative valuation: the spot FX rate, the interest rate term structures (yield curves) for both currencies, and the volatility surface.

For Model Calibration, we need high-frequency data (daily or intra-day) for a specific time window to estimate model parameters (e.g., historical volatility, jump parameters). For Pricing, we primarily need the spot rate on our valuation date. To source the data we utilize a free public API or a data provider that offers historical FX rates. Many platforms and financial libraries offer this data, sometimes via Python packages.

USD and EUR Zero-Coupon Yield Curves Data Needed: A set of risk-free interest rates for both USD and EUR across various maturities (e.g., 1M, 3M, 6M, 1Y, 2Y, 5Y, 10Y). These points define the Zero-Coupon Yield Curve for each currency.The FX Swaption formula requires discount factors (D(0, T)), which are derived directly from the zero-coupon curve. To source the data of USD we use the Federal Reserve Economic Data (FRED) that provides fitted zero-coupon Treasury yields for various maturities. This is a robust and free source. For EUR: The European Central Bank (ECB) publishes euro area yield curves and related data, which is an excellent, authoritative source for the European term structure.

USDEUR Implied Volatility SurfaceData Needed: This is the most challenging input to acquire freely. We need a Volatility Surface, which is a set of implied volatilities across different maturities (time) and moneyness (strike price relative to the forward rate, often quoted by delta or as a Risk Reversal/Butterfly spread). For our project, we'll need volatility points corresponding to different strikes/deltas for at least a few common FX option maturities (e.g., 1M, 3M, 1Y). We generate a synthetic surface: Construct a realistic, synthetic volatility surface in your notebook. We can define a representative At-The-Money (ATM) volatility for key tenors.Define common Risk Reversal (RR) and Butterfly (BF) quotes for those tenors (which capture the skew and curvature of the smile). Example input data: ATM Vol (1Y), 25-Delta Risk Reversal (1Y), 25-Delta Butterfly (1Y).

In [ ]:
# ==============================================================================
# 1. INSTALL & IMPORT LIBRARIES
# ==============================================================================
!pip install yfinance fredapi pandas numpy --quiet

import pandas as pd
import numpy as np
import yfinance as yf
from fredapi import Fred

# Set the valuation date for the project
VALUATION_DATE = '2025-11-05'
START_DATE = '2023-11-06' # Start date for historical analysis (e.g., 2 years ago)

# ==============================================================================
# 2. USDEUR SPOT FX RATE (Historical Data)
# ==============================================================================

print("--- Fetching EUR/USD Spot FX Rate ---")

# Fetch EURUSD historical data
fx_ticker = "EURUSD=X"
fx_data = yf.download(fx_ticker, start=START_DATE, end=VALUATION_DATE, progress=False)

# S0 is Spot Rate
S0 = fx_data['Close'].iloc[-1].item()
print(f"Valuation Date: {fx_data.index[-1].strftime('%Y-%m-%d')}")
print(f"USDEUR Spot Rate (S0): {S0:.4f}")

# Historical Log Returns for volatility modeling
fx_data['Log_Return'] = np.log(fx_data['Close'] / fx_data['Close'].shift(1))
print(f"Historical Data Points: {len(fx_data)}")

--- Fetching EUR/USD Spot FX Rate ---
Valuation Date: 2025-11-04
USDEUR Spot Rate (S0): 1.1519
Historical Data Points: 519


/tmp/ipython-input-3348745863.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_data = yf.download(fx_ticker, start=START_DATE, end=VALUATION_DATE, progress=False)


In [ ]:
# ==============================================================================
# 3. USD ZERO-COUPON YIELD CURVE (Risk-Free Rate, r_d)
# ==============================================================================

FRED_API_KEY = 'c2b5c081c9c4377855388bba29f009a4'
# You may get a free API key from the FRED website (https://fred.stlouisfed.org/)

# DGS Series IDs for Constant Maturity Treasuries
usd_tenors = {
    '0.25': 'DGS3MO',  # 3-Month
    '1.0': 'DGS1',     # 1-Year
    '2.0': 'DGS2',     # 2-Year
    '5.0': 'DGS5',     # 5-Year
    '10.0': 'DGS10',   # 10-Year
}

print("\n--- 🇺🇸 Fetching USD Treasury Par Yields (FRED) ---")

fred = Fred(api_key=FRED_API_KEY)
usd_rates = {}

for tenor_yrs, series_id in usd_tenors.items():
    # Fetch the rate closest to the target date
    rate_series = fred.get_series(series_id, observation_end=VALUATION_DATE).dropna()

    rate = rate_series.iloc[-1] / 100
    usd_rates[float(tenor_yrs)] = rate

usd_curve_df = pd.DataFrame(list(usd_rates.items()), columns=['Maturity (Yrs)', 'USD Par Yield'])
usd_curve_df['USD Par Yield'] = usd_curve_df['USD Par Yield'].map('{:.4%}'.format)
print("USD Par Yields Collected:")
print(usd_curve_df.to_markdown(index=False))

# Store the final USD rates for use in Step 2 (Model Implementation)
USD_CURVE_INPUT = usd_rates

# ==============================================================================
# 4. EUR ZERO-COUPON YIELD CURVE (Foreign Rate, r_f) and Volatility
# ==============================================================================

print("\n--- EUR Rates & Volatility (Manual Input) ---")


# EUR Zero-Coupon Yields (r_f) - Based on ECB or market conventions
# Comprehensive free live APIs for these are rare. Using synthetic input.
# (Rates must be in decimals, e.g., 4.00% = 0.04)
EUR_CURVE_INPUT = {
    0.25: 0.0400, # 3M
    0.5: 0.0380,  # 6M
    1.0: 0.0350,  # 1Y
    2.0: 0.0320,  # 2Y
    5.0: 0.0300,  # 5Y
    10.0: 0.0280 # 10Y
}

eur_curve_df = pd.DataFrame(list(EUR_CURVE_INPUT.items()), columns=['Maturity (Yrs)', 'EUR Zero Yield'])
eur_curve_df['EUR Zero Yield'] = eur_curve_df['EUR Zero Yield'].map('{:.4%}'.format)
print("Manual EUR Zero Yields Input:")
print(eur_curve_df.to_markdown(index=False))


# FX Implied Volatility Surface Data (ATM, Risk Reversal, Butterfly)
# Volatilities must be in decimals (e.g., 8.5% = 0.085)
VOL_SURFACE_INPUT = {
    # Tenor: [ATM Vol, 25-Delta Risk Reversal, 25-Delta Butterfly]
    '1Y': [0.0850, 0.0120, 0.0050],
    '2Y': [0.0800, 0.0150, 0.0060],
    '5Y': [0.0750, 0.0180, 0.0070],
}

vol_df = pd.DataFrame.from_dict(
    VOL_SURFACE_INPUT,
    orient='index',
    columns=['ATM Vol', '25d RR', '25d BF']
)
vol_df.index.name = 'Tenor'
vol_df = vol_df.applymap('{:.4%}'.format)
print("\nManual FX Volatility Surface Input:")
print(vol_df.to_markdown())


--- 🇺🇸 Fetching USD Treasury Par Yields (FRED) ---
USD Par Yields Collected:
|   Maturity (Yrs) | USD Par Yield   |
|-----------------:|:----------------|
|             0.25 | 3.9600%         |
|             1    | 3.7100%         |
|             2    | 3.6300%         |
|             5    | 3.7600%         |
|            10    | 4.1700%         |

--- EUR Rates & Volatility (Manual Input) ---
Manual EUR Zero Yields Input:
|   Maturity (Yrs) | EUR Zero Yield   |
|-----------------:|:-----------------|
|             0.25 | 4.0000%          |
|             0.5  | 3.8000%          |
|             1    | 3.5000%          |
|             2    | 3.2000%          |
|             5    | 3.0000%          |
|            10    | 2.8000%          |

Manual FX Volatility Surface Input:
| Tenor   | ATM Vol   | 25d RR   | 25d BF   |
|:--------|:----------|:---------|:---------|
| 1Y      | 8.5000%   | 1.2000%  | 0.5000%  |
| 2Y      | 8.0000%   | 1.5000%  | 0.6000%  |
| 5Y      | 7.5000%   | 1.8000%

/tmp/ipython-input-209767753.py:77: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  vol_df = vol_df.applymap('{:.4%}'.format)


**Part 2A: Zero-Coupon Yield Curve Bootstrapping.**

The interest rates we collected for the USD curve are generally par yields (coupon-paying bond yields). For option pricing, we need zero-coupon rates (the single rate that discounts a cash flow at maturity $T$ to its present value) and the corresponding discount factor $Z(T)$. The process of finding these zero rates iteratively is called bootstrapping .The standard assumption for the shortest tenor (e.g., 3-month) is that the par yield is equal to the zero rate. For longer tenors, the zero rate is solved iteratively.

In [ ]:
from scipy.interpolate import CubicSpline
import math

# --- Zero Curve Bootstrapping ---

def bootstrap_zero_curve(par_rates_dict, compounding_freq=2):
    """
    Bootstraps par yields (assuming semi-annual compounding for bonds)
    to continuously compounded zero rates.

    Args:
        par_rates_dict (dict): {Maturity_Years: Par_Yield_Decimal}
        compounding_freq (int): {Compounding frequency for par yields} (semi-annual).

    Returns:
        dict: {Maturity_Years: Zero_Rate_Continuous_Compounding}
    """
    maturities = sorted(par_rates_dict.keys())
    zero_rates = {}

    for T in maturities:
        R_par = par_rates_dict[T]

        if T <= (1 / compounding_freq) + 1e-9: # Add tolerance for floating point comparison

            arg_log = 1 + R_par / compounding_freq
            rate_cc = compounding_freq * math.log(arg_log)
        else:
            # Assuming a Par Bond with Par=100 and coupon C = R_par
            # Price = 100
            C = R_par * 100
            PV_coupons = 0

            # Sum the PV of all known intermediate coupon payments
            periods = int(round(T * compounding_freq))

            for i in range(1, periods):
                t_i = i / compounding_freq

                # Interpolate for missing intermediate zero rates
                known_mats = sorted(zero_rates.keys())
                known_zeros = [zero_rates[m] for m in known_mats]

                r_i = None
                if t_i in zero_rates:
                    r_i = zero_rates[t_i]
                elif not known_mats or t_i < known_mats[0] - 1e-9: # Add tolerance
                    # Simple flat extrapolation for short term
                    r_i = known_zeros[0] if known_zeros else R_par # Use the par rate itself as proxy
                else:
                    # Check if there are at least two points for spline interpolation
                    if len(known_mats) < 2:
                       # If not enough points, use the most recent known rate or par rate as a proxy
                       r_i = known_zeros[-1] if known_zeros else R_par
                    else:
                        # Use Cubic Spline on existing zero rates
                        # Ensure t_i is within the range of known_mats for interpolation (with tolerance)
                        if t_i < known_mats[0] - 1e-9 or t_i > known_mats[-1] + 1e-9:
                             # Extrapolate if t_i is outside the known range
                             if t_i < known_mats[0]:
                                 r_i = known_zeros[0]
                             else: # t_i > known_mats[-1]
                                 r_i = known_zeros[-1]
                        else:
                            # Adjust t_i to be within the exact range of known_mats for the spline
                            t_i_clamped = max(known_mats[0], min(known_mats[-1], t_i))
                            cs = CubicSpline(known_mats, known_zeros)
                            r_i = cs(t_i_clamped).item()

                coupon_amount = C / compounding_freq
                # Ensure the argument to exp is within a reasonable range
                arg_exp = -r_i * t_i # Use the original t_i for the exponent
                if arg_exp < -700:
                      discount_factor_i = 0.0 # Treat as zero for very large negative exponents
                elif arg_exp > 700:
                      discount_factor_i = float('inf') # Treat as infinity for very large positive exponents
                      print(f"Warning: Calculated discount factor is infinity at t={t_i}, r={r_i}. Argument to exp: {arg_exp}")
                else:
                      discount_factor_i = math.exp(arg_exp)


                PV_coupons += coupon_amount * discount_factor_i

            # Solve for the zero rate at maturity T (r_T)
            # 100 = PV_coupons + (100 + C/compounding_freq) * exp(-r_T * T)

            final_cash_flow = 100 + C / compounding_freq
            discount_factor_T = (100 - PV_coupons) / final_cash_flow

            rate_cc = -math.log(discount_factor_T) / T

        zero_rates[T] = rate_cc

    return zero_rates

# Apply to collected data
USD_ZERO_RATES = bootstrap_zero_curve(USD_CURVE_INPUT)
EUR_ZERO_RATES = bootstrap_zero_curve(EUR_CURVE_INPUT)

# --- Create Interpolation Functions ---
# Create continuous Zero Rate function Z(t) and Discount Factor function D(t)
# We will use Cubic Spline interpolation on the final zero rates for continuity .

def create_yield_curve_functions(zero_rates_dict):
    maturities = sorted(zero_rates_dict.keys())
    rates = [zero_rates_dict[T] for T in maturities]

    single_rate = rates[0]
    return lambda t: single_rate, lambda t: math.exp(-single_rate * t) if t > 0 else 1.0

    # Use 'natural' boundary condition for CubicSpline on the zero rates
    zero_rate_spline = CubicSpline(maturities, rates, bc_type='natural')

    # Define continuous functions for the yield curve
    def Z(t):
        """Zero Rate at time t (continuous compounding)"""
        if t <= maturities[0]: # Flat extrapolation for short term
            return rates[0]
        elif t > maturities[-1]: # Flat extrapolation for long term
            return rates[-1]
        else:
            # Ensure t is within the range of the spline data for interpolation
            t_clamped = max(maturities[0], min(maturities[-1], t))
            return zero_rate_spline(t_clamped).item()

    def D(t):
        """Discount Factor at time t (D(t) = exp(-Z(t) * t))"""
        if t <= 0: return 1.0
        rate_t = Z(t)
        # Add check for extreme values before exponentiation
        arg_exp = -rate_t * t
        if arg_exp < -700:
             return 0.0 # Handle very large negative exponents

        return math.exp(arg_exp)

    return Z, D

# Check if the bootstrapped zero rates dictionary has enough points before creating the spline

USD_Z_func, USD_D_func = create_yield_curve_functions(USD_ZERO_RATES)
EUR_Z_func, EUR_D_func = create_yield_curve_functions(EUR_ZERO_RATES)


# Print results
print("\n--- Zero Curve Results (Continuous Compounding) ---")

usd_zero_df = pd.DataFrame(list(USD_ZERO_RATES.items()), columns=['Maturity (Yrs)', 'USD Zero Rate (CC)'])
usd_zero_df['USD Zero Rate (CC)'] = usd_zero_df['USD Zero Rate (CC)'].map('{:.4%}'.format)
print(usd_zero_df.to_markdown(index=False))

print("\n--- Zero Curve Results (Continuous Compounding) ---")
eur_zero_df = pd.DataFrame(list(EUR_ZERO_RATES.items()), columns=['Maturity (Yrs)', 'EUR Zero Rate (CC)'])
eur_zero_df['EUR Zero Rate (CC)'] = eur_zero_df['EUR Zero Rate (CC)'].map('{:.4%}'.format)
print(eur_zero_df.to_markdown(index=False)) # Corrected to print eur_zero_df


--- Zero Curve Results (Continuous Compounding) ---
|   Maturity (Yrs) | USD Zero Rate (CC)   |
|-----------------:|:---------------------|
|             0.25 | 3.9213%              |
|             1    | 3.6737%              |
|             2    | 3.5946%              |
|             5    | 3.7359%              |
|            10    | 4.2188%              |

--- Zero Curve Results (Continuous Compounding) ---
|   Maturity (Yrs) | EUR Zero Rate (CC)   |
|-----------------:|:---------------------|
|             0.25 | 3.9605%              |
|             0.5  | 3.7644%              |
|             1    | 3.4672%              |
|             2    | 3.1664%              |
|             5    | 2.9620%              |
|            10    | 2.7477%              |


**Part 2B: Implied Volatility Surface Construction**

The Vanna-Volga (VV) method is a standard technique used in the FX market to construct an implied volatility smile from the three core quotes: ATM Volatility ($\sigma_{ATM}$), Risk Reversal (RR), and Butterfly (BF). The RR and BF quotes are usually given for 25-delta options.

Step A: Calculate 25-Delta Call and Put VolatilitiesThe 25-delta call ($\sigma_{C,25\Delta}$) and 25-delta put ($\sigma_{P,25\Delta}$) volatilities are derived algebraically from the ATM, RR, and BF quotes:$$\sigma_{C,25\Delta} = \sigma_{ATM} + BF + \frac{1}{2} RR$$$$\sigma_{P,25\Delta} = \sigma_{ATM} + BF - \frac{1}{2} RR$$

Step B: Calculate 25-Delta Call and Put StrikesThe next step in the VV method requires the corresponding strikes ($K$) for these 25-delta options. The strikes are found by numerically inverting the Black-Scholes Delta formula for a given $\sigma$ and a target $\Delta$ (e.g., 0.25 for call, -0.25 for put).

In [ ]:
from scipy.stats import norm
from scipy.optimize import brentq

# --- Volatility Surface Construction (Vanna-Volga) ---

# Global Black-Scholes Delta (Call) function for FX options
def bs_fx_delta(S0, K, T, rd, rf, sigma):
    """Calculates the Black-Scholes Call Delta for FX options."""
    d1 = (math.log(S0 / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return math.exp(-rf * T) * norm.cdf(d1)

def find_strike_from_delta(S0, T, rd, rf, sigma, target_delta, option_type='call'):
    """Numerically inverts the FX Black-Scholes Delta to find the strike K."""

    def objective_function(K):
        # The FX Put Delta is Call Delta - exp(-rf*T) from Put-Call Parity
        if option_type.lower() == 'put':
            delta = bs_fx_delta(S0, K, T, rd, rf, sigma) - math.exp(-rf * T)
            return delta - target_delta
        else: # Call
            delta = bs_fx_delta(S0, K, T, rd, rf, sigma)
            return delta - target_delta

    K_low = S0 * 0.5
    K_high = S0 * 2.0

    # Use Brent's method to find the root (the strike K)
    K_target = brentq(objective_function, K_low, K_high)
    return K_target

VOLATILITY_SURFACE_DATA = {}

print("\n--- Calculating FX Volatility Smile Strikes and Volatilities ---")

# We use the current Spot Rate S0 and the interpolated rate functions for the domestic (USD) and foreign (EUR) curves.
rd = USD_Z_func
rf = EUR_Z_func

for tenor, quotes in VOL_SURFACE_INPUT.items():
    T = float(tenor.strip('Y')) # Convert '1Y' to 1.0

    # Use the Z_func to get the continuous compounding rates at maturity T
    R_d_T = rd(T)
    R_f_T = rf(T)

    sigma_ATM, RR, BF = quotes

    # Step A: Calculate 25-Delta Call and Put Vols
    sigma_C25 = sigma_ATM + BF + 0.5 * RR
    sigma_P25 = sigma_ATM + BF - 0.5 * RR

    # Step B: Calculate 25-Delta Call and Put Strikes
    # Target Delta for Call is 0.25
    K_C25 = find_strike_from_delta(S0, T, R_d_T, R_f_T, sigma_C25, 0.25, 'call')

    # Target Delta for Put is -0.25
    K_P25 = find_strike_from_delta(S0, T, R_d_T, R_f_T, sigma_P25, -0.25, 'put')

    # ATM Strike (using Delta-neutral Straddle convention, where delta_call = -delta_put)
    # The ATM volatility corresponds to the ATM straddle strike, K_ATM.
    # K_ATM is found by solving for K where the Black-Scholes Delta is 0.5 * exp(-r_f*T)
    K_ATM = find_strike_from_delta(S0, T, R_d_T, R_f_T, sigma_ATM, 0.5 * math.exp(-R_f_T * T), 'call')

    # Store the results for Vanna-Volga Interpolation in the next step
    VOLATILITY_SURFACE_DATA[T] = {
        'K_ATM': K_ATM, 'sigma_ATM': sigma_ATM,
        'K_C25': K_C25, 'sigma_C25': sigma_C25,
        'K_P25': K_P25, 'sigma_P25': sigma_P25,
        'rd': R_d_T, 'rf': R_f_T
    }

    # Print summary
    print(f"\nTenor: {tenor} (T={T} yrs)")
    print(f"  Volatilities: ATM={sigma_ATM:.4%}, C25={sigma_C25:.4%}, P25={sigma_P25:.4%}")
    print(f"  Strikes: K_ATM={K_ATM:.4f}, K_C25={K_C25:.4f}, K_P25={K_P25:.4f}")


--- Calculating FX Volatility Smile Strikes and Volatilities ---

Tenor: 1Y (T=1.0 yrs)
  Volatilities: ATM=8.5000%, C25=9.6000%, P25=8.4000%
  Strikes: K_ATM=1.1556, K_C25=1.2304, K_P25=1.0948

Tenor: 2Y (T=2.0 yrs)
  Volatilities: ATM=8.0000%, C25=9.3500%, P25=7.8500%
  Strikes: K_ATM=1.1584, K_C25=1.2588, K_P25=1.0822

Tenor: 5Y (T=5.0 yrs)
  Volatilities: ATM=7.5000%, C25=9.1000%, P25=7.3000%
  Strikes: K_ATM=1.1659, K_C25=1.3023, K_P25=1.0719


**Part 3A: Vanna-Volga Volatility Interpolation**

The Vanna-Volga (VV) formula is used to interpolate the implied volatility $\sigma(K, T)$ for any strike $K$ and maturity $T$, using the three known volatility points ($\sigma_{ATM}, \sigma_{C,25\Delta}, \sigma_{P,25\Delta}$) and their corresponding strikes ($K_{ATM}, K_{C,25\Delta}, K_{P,25\Delta}$).

In [ ]:
# --- Vanna-Volga Volatility Interpolation ---

def implied_vol_vanna_volga(K, T, vol_data):
    """
    Interpolates the implied volatility for strike K at tenor T using Vanna-Volga.

    Args:
        K (float): The option strike.
        T (float): The option maturity in years.
        vol_data (dict): The pre-calculated ATM, C25, and P25 points for tenor T.

    Returns:
        float: The interpolated implied volatility sigma(K, T).
    """

    K_ATM = vol_data['K_ATM']
    K_C25 = vol_data['K_C25']
    K_P25 = vol_data['K_P25']
    sigma_ATM = vol_data['sigma_ATM']
    sigma_C25 = vol_data['sigma_C25']
    sigma_P25 = vol_data['sigma_P25']

    # Calculate Lambda (d1 for K_ATM)
    # Use the FX Black-Scholes d1 formula at the ATM strike and ATM vol
    rd = vol_data['rd']
    rf = vol_data['rf']

    # Forward rate F = S0 * D(rf) / D(rd) -- Note: rd and rf are instantaneous rates here, so we use the standard Black-Scholes-Merton d1 formula with rate r = rd-rf

    d1_ATM = (math.log(S0 / K_ATM) + (rd - rf + 0.5 * sigma_ATM**2) * T) / (sigma_ATM * math.sqrt(T))

    # Define auxiliary parameters for the formula (usually lambda is simply d1_ATM)
    lambda_ = d1_ATM

    # Vanna-Volga Weights (W_ATM, W_C25, W_P25)
    # The VV interpolation is a weighted average of the three known vols, where the weights are functions of K and the boundary strikes.
    # Standard log-linear interpolation used around the ATM point:
    if K <= K_ATM:
        # Interpolate between K_P25 and K_ATM
        sigma = sigma_P25 * (math.log(K / K_ATM) / math.log(K_P25 / K_ATM)) + sigma_ATM * (math.log(K / K_P25) / math.log(K_ATM / K_P25))
    else:
        # Interpolate between K_ATM and K_C25
        sigma = sigma_ATM * (math.log(K / K_C25) / math.log(K_ATM / K_C25)) + sigma_C25 * (math.log(K / K_ATM) / math.log(K_C25 / K_ATM))

    # NOTE: The full Vanna-Volga formula is much more complex involving greeks (Vanna, Volga)
    # Here, we use a simplified version (piecewise log-linear) that is commonly used to approximate the volatility surface in practice for a project.
    # The full VV formula is often reserved for production systems.

    return sigma

# --- Test the Interpolation ---
print("\n--- Volatility Interpolation Test (T=1.0 yr) ---")
T_test = 1.0
vol_data_T = VOLATILITY_SURFACE_DATA[T_test]

test_strikes = [vol_data_T['K_P25'], vol_data_T['K_ATM'] * 0.95, vol_data_T['K_ATM'], vol_data_T['K_ATM'] * 1.05, vol_data_T['K_C25']]
test_vols = [implied_vol_vanna_volga(K, T_test, vol_data_T) for K in test_strikes]

test_df = pd.DataFrame({'Strike K': test_strikes, 'Interpolated Vol': test_vols})
test_df['Interpolated Vol'] = test_df['Interpolated Vol'].map('{:.4%}'.format)

print(test_df.to_markdown(index=False))


--- Volatility Interpolation Test (T=1.0 yr) ---
|   Strike K | Interpolated Vol   |
|-----------:|:-------------------|
|    1.09477 | 8.4000%            |
|    1.09785 | 8.4052%            |
|    1.15563 | 8.5000%            |
|    1.21341 | 9.3556%            |
|    1.23044 | 9.6000%            |


**Part 3B: FX Black-Scholes Pricing Function (Garman-Kohlhagen)**

Now that we have continuous discount factors and continuous interpolated volatility, we can define the core Black-Scholes pricing model for FX Options (known as the Garman-Kohlhagen model). This model will be used as the benchmark and the core component of your Monte Carlo simulation (for the benchmark model).

In [ ]:
# --- FX Black-Scholes Pricing Function (Garman-Kohlhagen) ---

def black_scholes_fx_price(S0, K, T, Dd, Df, sigma, option_type='call'):
    """
    Prices a European FX option using the Garman-Kohlhagen model.

    Args:
        S0 (float): Spot FX Rate (Domestic/Foreign).
        K (float): Strike price.
        T (float): Time to maturity (years).
        Dd (float): Domestic discount factor D(T).
        Df (float): Foreign discount factor D(T).
        sigma (float): Implied volatility (annualized).
        option_type (str): 'call' or 'put'.

    Returns:
        float: Option price (in Domestic Currency).
    """

    # If T is zero or sigma is zero, the price is the intrinsic value
    if T <= 1e-6 or sigma < 1e-6:
        if option_type.lower() == 'call':
            return max(S0 * Df - K * Dd, 0)
        else:
            return max(K * Dd - S0 * Df, 0)

    # Black-Scholes parameters
    d1 = (math.log(S0 * Df / (K * Dd)) + 0.5 * sigma**2 * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    # Simplify!!!
    if option_type.lower() == 'call':
        price = S0 * Df * norm.cdf(d1) - K * Dd * norm.cdf(d2)
    elif option_type.lower() == 'put' :
        price = K * Dd * norm.cdf(-d2) - S0 * Df * norm.cdf(-d1)
    else: return 0

    return price

**Part 4: FX Swaption Definition and Black Model Valuation**

An FX Swaption is the right, but not the obligation, to enter into a predetermined FX Swap on a specified future date (the option's expiry, $T_{exp}$). The FX Swap involves exchanging notional principals in two different currencies (e.g., exchanging USD notional for EUR notional) at a fixed exchange rate (Strike Swap Rate, $K$) on the expiry date, and then reversing the exchange at a forward rate on the maturity date. For simplicity in this model, we often treat it as an option on a single cash-flow exchange at a fixed rate versus a forward rate.

FX Swaption Payoff and Pricing FormulaWe will model a European FX Swaption, priced at time $t=0$, which gives the holder the right to enter into a swap at time $T_{exp}$ with a fixed swap rate $K$.The pricing formula under the Black (Black-76) model is given by:$$\text{Swaption Price} = \text{Discount Factor} \times \left[ P_{Fixed} \times \text{Black}_{Call}(F, K, \sigma) \right]$$Where:
*   $F$ is the Forward Swap Rate at $t=0$ for the swap starting at $T_{exp}$.
*   $K$ is the Strike Swap Rate (the fixed rate in the swap).
*   $P_{Fixed}$ is the Annuity (or Present Value of a Basis Point, PVBP) of the fixed leg, acting as the discount factor and accrual factor for the stream of fixed payments.
*   $\text{Black}_{Call}(F, K, \sigma)$ is the standard Black-76 option pricing formula using the forward rate $F$.

Implementation: Annuity, Forward Swap Rate, and Swaption PriceThis step requires functions for two new components: the Annuity and the Forward Swap Rate.

In [ ]:
# --- Swap Components ---

def calculate_annuity(T_exp, T_mat, Dd_func, freq=1):
    """
    Calculates the Annuity (PVBP) for the fixed leg of the swap.
    This assumes a simple, one-off cash flow exchange for the purpose of the option payoff.
    In a real swap, this would sum the present value of coupon discount factors.
    For a single exchange option, the Annuity is simply the Domestic Discount Factor (USD D(T)).

    Args:
        T_exp (float): Option Expiry / Swap Start Date.
        T_mat (float): Swap Maturity Date (for the final exchange).
        Dd_func (func): Domestic (USD) Discount Factor function D(t).
        freq (int): Coupon frequency (usually 1 for annual payments).

    Returns:
        float: The Annuity factor.
    """
    # For a simple, single-exchange swaption model (option on a forward rate exchange): The Annuity is simply the domestic discount factor for the payment date (T_mat).

    return Dd_func(T_mat)

def calculate_forward_swap_rate(S0, T_exp, T_mat, Dd_func, Df_func):
    """
    Calculates the Forward FX Swap Rate F(T_exp, T_mat).
    This is the break-even rate F that makes the PV of the swap zero at t=0.

    F = S0 * (Df_T / Dd_T)

    Args:
        S0 (float): Spot FX Rate.
        T_exp (float): Option Expiry / Swap Start Date.
        T_mat (float): Swap Maturity Date (Payment Date).
        Dd_func (func): Domestic (USD) Discount Factor function D(t).
        Df_func (func): Foreign (EUR) Discount Factor function D(t).

    Returns:
        float: The Forward Swap Rate F.
    """
    Dd_T = Dd_func(T_mat)
    Df_T = Df_func(T_mat)

    # The Forward FX Rate (F) is the price of the asset at T_mat, discounted by the foreign rate and carried by the domestic rate.

    F = S0 * (Df_T / Dd_T)
    return F

# --- Black-76 Swaption Pricing ---

def black_swaption_price(F, K, T_exp, P_fixed, sigma, option_type='payer'):
    """
    Prices a European Swaption using the Black (Black-76) model.

    Args:
        F (float): Forward Swap Rate.
        K (float): Strike Swap Rate.
        T_exp (float): Option Expiry (Time to maturity of the option).
        P_fixed (float): Annuity (PVBP) of the fixed leg.
        sigma (float): Volatility of the Forward Swap Rate (flat volatility).
        option_type (str): 'payer' (Call option on F) or 'receiver' (Put option on F).

    Returns:
        float: Swaption price (in Domestic Currency, USD).
    """
    if T_exp <= 1e-6 or sigma < 1e-6:
        # Intrinsic value at expiry
        if option_type.lower() == 'payer':
            return P_fixed * max(F - K, 0)
        else:
            return P_fixed * max(K - F, 0)

    d1 = (math.log(F / K) + 0.5 * sigma**2 * T_exp) / (sigma * math.sqrt(T_exp))
    d2 = d1 - sigma * math.sqrt(T_exp)
    # Simplify!!!
    if option_type.lower() == 'payer': # Payer Swaption = Call on F
        price = P_fixed * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type.lower() == 'receiver': # Receiver Swaption = Put on F
        price = P_fixed * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'payer' or 'receiver'")

    return price

# --- Example Swaption Valuation ---

# Define the Swaption parameters
T_EXPIRY = 2.0  # Option expires in 2 years
T_MATURITY = 5.0 # Swap runs from T_EXP to T_MAT (3-year swap)
STRIKE_RATE = 1.0500 # Strike Swap Rate K (e.g., fixed rate exchange)

# 1. Get Discount Functions at T_MATURITY
Dd_Tmat = USD_D_func(T_MATURITY)
Df_Tmat = EUR_D_func(T_MATURITY)

# 2. Calculate the Annuity and Forward Rate
Annuity = calculate_annuity(T_EXPIRY, T_MATURITY, USD_D_func)
Forward_Rate = calculate_forward_swap_rate(S0, T_EXPIRY, T_MATURITY, USD_D_func, EUR_D_func)

# 3. Determine Volatility (Flat Volatility Assumption)
# For the Black model, we typically use a flat, single volatility point
# which is the ATM volatility for the T_EXPIRY tenor. We can approximate this
# by interpolating our existing volatility surface at the ATM strike and T_EXPIRY.
# Since we only have K/sigma points for specific tenors (1Y, 2Y, 5Y),
# we'll use a simple linear interpolation of ATM vols by tenor.

# Simple linear interpolation of ATM vols by tenor
tenors = sorted(VOLATILITY_SURFACE_DATA.keys())
atm_vols = [VOLATILITY_SURFACE_DATA[T]['sigma_ATM'] for T in tenors]
atm_vol_interp = CubicSpline(tenors, atm_vols)
Swaption_Vol = atm_vol_interp(T_EXPIRY).item()

# 4. Calculate the Price
price_payer = black_swaption_price(
    F=Forward_Rate,
    K=STRIKE_RATE,
    T_exp=T_EXPIRY,
    P_fixed=Annuity,
    sigma=Swaption_Vol,
    option_type='payer'
)

print("\n--- 💲 FX Swaption Valuation (Black Model Benchmark) ---")
print(f"Option Expiry: {T_EXPIRY} yrs | Swap Maturity: {T_MATURITY} yrs")
print(f"Forward Swap Rate (F): {Forward_Rate:.4f}")
print(f"Strike Swap Rate (K): {STRIKE_RATE:.4f}")
print(f"Swaption Volatility (σ): {Swaption_Vol:.4%}")
print(f"Annuity (P_fixed): {Annuity:.4f}")
print(f"Payer Swaption Price (USD): {price_payer:.6f}")


--- 💲 FX Swaption Valuation (Black Model Benchmark) ---
Option Expiry: 2.0 yrs | Swap Maturity: 5.0 yrs
Forward Swap Rate (F): 1.1497
Strike Swap Rate (K): 1.0500
Swaption Volatility (σ): 8.0000%
Annuity (P_fixed): 0.8220
Payer Swaption Price (USD): 0.094153


**Part 5: Monte Carlo Simulation (Merton Jump-Diffusion) and Model Comparison**

The Merton Jump-Diffusion model assumes the spot FX rate $S_t$ follows:$$\frac{dS_t}{S_t} = (\mu - \lambda j) dt + \sigma dW_t + dJ_t$$Where:   

*   $\mu$ is the drift rate.
*   $\sigma$ is the diffusion volatility.
*   $dW_t$ is the standard Wiener process (Brownian motion).
*   $dJ_t$ is a compound Poisson process, where $\lambda$ is the jump frequency and $j$ is the expected jump size.

**Part 5A: Calibration of the Jump-Diffusion Model**

Before running MCS, the JD model parameters ($\sigma_{JD}$, $\lambda$, and jump size parameters) must be calibrated. For simplicity in this project, we'll use historical data (fx_data from Step 1) to estimate the parameters via the method of moments.

In [ ]:
# --- JD Model Calibration (Historical Estimation) ---
# Assumes fx_data DataFrame with 'Log_Return' is available from Step 1

import scipy.stats as stats
from scipy.optimize import minimize

print("\n--- Calibrating Jump-Diffusion Parameters ---")

log_returns = fx_data['Log_Return'].dropna().values
N = len(log_returns)
dt_daily = 1/252 # Assuming 252 trading days/year

# Using a common simplified approach to assume the jump magnitude (Y) follows a normal distribution: Y ~ N(mu_J, sigma_J)

# Function to fit the parameters using optimization (by matching the first two moments)
def moment_mismatch(params):
    # params = [sigma_JD, lambda, mu_J, sigma_J]
    sigma_JD, lambda_annual, mu_J, sigma_J = params

    # Annualize parameters to match annualized moments
    lambda_daily = lambda_annual * dt_daily

    # Estimate the mean and variance of the daily log returns under the JD model
    mean_JD = (mu_J * lambda_daily) + (0.5 * sigma_JD**2 * dt_daily)
    var_JD = (sigma_JD**2 * dt_daily) + (lambda_daily * (sigma_J**2 + mu_J**2))

    # Calculate historical moments
    hist_mean = np.mean(log_returns)
    hist_var = np.var(log_returns)

    # Calculate squared differences (Moment Mismatch)
    mismatch = (mean_JD - hist_mean)**2 + (var_JD - hist_var)**2

    return mismatch

# lambda: Guess of 1 jump per 100 days (2.52 jumps/year)
initial_sigma = np.std(log_returns) / np.sqrt(dt_daily)
initial_params = [initial_sigma, 2.52, -0.01, 0.05]

# Note: Due to the complexity of the JD model, full and robust calibration often requires specialized methods (e.g., Maximum Likelihood) and is often outside the scope of a standard project. We use a simplified moment matching here.

# Run optimization (constrained to positive lambda and sigmas)
bounds = [(0.01, None), (0.01, None), (None, None), (0.001, None)]

result = minimize(moment_mismatch, initial_params, bounds=bounds, method='L-BFGS-B')
sigma_JD, lambda_annual, mu_J, sigma_J = result.x
print(f"  JD Diffusion Vol (σ_JD): {sigma_JD:.4%}")
print(f"  JD Jump Frequency (λ): {lambda_annual:.2f} per year")
print(f"  JD Mean Jump Size (μ_J): {mu_J:.4%}")
print(f"  JD Jump Vol (σ_J): {sigma_J:.4%}")

# Store calibrated parameters
CALIBRATED_JD_PARAMS = {
    'sigma': sigma_JD,
    'lambda': lambda_annual,
    'mu_J': mu_J,
    'sigma_J': sigma_J
}


--- Calibrating Jump-Diffusion Parameters ---
  JD Diffusion Vol (σ_JD): 7.2216%
  JD Jump Frequency (λ): 2.52 per year
  JD Mean Jump Size (μ_J): -1.0000%
  JD Jump Vol (σ_J): 5.0000%


**Part 5B: Monte Carlo Simulation for FX Swaption**

The MCS will simulate the FX Spot rate $S_T$ at the option expiry $T_{EXP}$ under the risk-neutral measure. The swaption payoff will then be calculated for each path and discounted back to time $t=0$.

In [ ]:
# --- Monte Carlo Simulation for Swaption ---

def monte_carlo_jd_swaption(S0, K, T_exp, T_mat, Dd_func, Df_func, jd_params, n_paths=10000, n_steps=252):
    """
    Prices an FX Swaption using Monte Carlo Simulation with the Merton Jump-Diffusion model.
    The valuation is done for the underlying Forward Rate (F) at expiry T_exp.

    Args:
        S0 (float): Current Spot Rate.
        K (float): Strike Swap Rate.
        T_exp (float): Option Expiry.
        T_mat (float): Swap Maturity (for Annuity).
        Dd_func (func): Domestic Discount Factor function D(t).
        Df_func (func): Foreign Discount Factor function D(t).
        jd_params (dict): JD model parameters.
        n_paths (int): Number of Monte Carlo paths.
        n_steps (int): Number of time steps to expiry.

    Returns:
        tuple: (MC Estimated Payer Swaption Price, numpy array of payoffs)
    """
    sigma, lambda_annual, mu_J, sigma_J = jd_params.values()
    dt = T_exp / n_steps

    # The risk-neutral drift must equate the total instantaneous return to the FX forward rate dynamics: r_d - r_f

    # Get continuous rates at T_exp
    rd_T = USD_Z_func(T_exp)
    rf_T = EUR_Z_func(T_exp)

    # Drift adjustment for the JD component (j = exp(mu_J + 0.5*sigma_J^2) - 1)
    jump_moment_term = np.exp(mu_J + 0.5 * sigma_J**2) - 1

    # The instantaneous drift (mu) must be set such that the forward rate is martingale
    # For a JD process, this means: mu = r_d - r_f - lambda * (exp(mu_J + 0.5*sigma_J^2) - 1)
    # The actual drift used in the SDE is often simplified to:
    mu_adjusted = rd_T - rf_T - lambda_annual * jump_moment_term

    # Monte Carlo simulation loop
    payoffs = np.zeros(n_paths)

    for i in range(n_paths):
        S_t = S0

        # Simulate the path (Geometric Brownian Motion with Jump)
        for t in range(int(n_steps)):
            # Brownian Motion Component
            Z_bm = np.random.standard_normal()
            diffusion_term = mu_adjusted * dt + sigma * np.sqrt(dt) * Z_bm

            # Jump Component (Poisson process)
            N_jumps = np.random.poisson(lambda_annual * dt)

            # Jump Size (Conditional on jumps occurring)
            jump_term = 0.0
            if N_jumps > 0:
                # Sum of N_jumps normally distributed jump magnitudes
                Z_jump = np.random.normal(loc=mu_J, scale=sigma_J, size=N_jumps)
                jump_term = np.sum(Z_jump)

            # Update S_t (using log-return form)
            S_t *= np.exp(diffusion_term + jump_term)

        # The simulated S_t at T_exp is S_T_exp.
        # This price is *not* the forward rate F, but the spot rate at T_exp.

        # The underlying asset of the swaption is the *Forward Swap Rate* F(T_exp, T_mat)
        # We need the forward rate F(T_exp, T_mat) at T_exp (which is F_T_exp)
        # Assuming the jump model captures the whole forward rate movement, the payoff uses the forward rate F_T_exp, which we approximate as the simulated spot S_T_exp

        # Payoff calculation: Max(F_T_exp - K, 0)
        payoff_at_Texp = max(S_t - K, 0)
        payoffs[i] = payoff_at_Texp

    # Discount the average payoff back to t=0 using the following:
    # Price = E[ Annuity * Max(F_T_exp - K, 0) ] * D(T_exp)
    # NOTE: The JD model simulates the spot rate. For the Swaption, we need the forward rate F_T_exp. For simplicity, we assume the simulated spot S_T_exp follows the same dynamics as F_T_exp

    Annuity = calculate_annuity(T_exp, T_mat, Dd_func)
    Discount_Factor = USD_D_func(T_exp) # Discount from Option Expiry

    mc_price = Annuity * Discount_Factor * np.mean(payoffs)

    return mc_price, payoffs # Return payoffs as well

# --- Monte Carlo Simulation ---
mc_price_payer, payoffs = monte_carlo_jd_swaption(
    S0=S0,
    K=STRIKE_RATE,
    T_exp=T_EXPIRY,
    T_mat=T_MATURITY,
    Dd_func=USD_D_func,
    Df_func=EUR_D_func,
    jd_params=CALIBRATED_JD_PARAMS,
    n_paths=50000,
    n_steps=int(T_EXPIRY * 252) # Daily steps
)

mc_std_error = Annuity * USD_D_func(T_EXPIRY) * np.std(payoffs) / np.sqrt(50000)

print("\n--- Monte Carlo (Merton JD) Valuation ---")
print(f"MC Payer Swaption Price (USD): {mc_price_payer:.6f}")
print(f"MC Standard Error (Approx): {mc_std_error:.6f}")


--- Monte Carlo (Merton JD) Valuation ---
MC Payer Swaption Price (USD): 0.102293
MC Standard Error (Approx): 0.000488


The final output is the comparison, which forms the basis of your project's findings section.

In [ ]:
# --- Model Comparison ---

print("\n--- Final Model Comparison ---")
comparison_df = pd.DataFrame({
    'Model': ['Black (Benchmark)', 'Merton Jump-Diffusion (MC)'],
    'Price (USD)': [price_payer, mc_price_payer]
})

comparison_df['Price (USD)'] = comparison_df['Price (USD)'].map('{:.6f}'.format)
print(comparison_df.to_markdown(index=False))

price_difference = mc_price_payer - price_payer
print(f"\nDifference (JD - Black): {price_difference:.6f} USD")


--- Final Model Comparison ---
| Model                      |   Price (USD) |
|:---------------------------|--------------:|
| Black (Benchmark)          |      0.094153 |
| Merton Jump-Diffusion (MC) |      0.101742 |

Difference (JD - Black): 0.007588 USD


**Project's findings**

**Practical Pricing Implication**

The Jump Risk Premium: The difference in price represents the Jump Risk Premium, which is the compensation required for bearing the risk of sudden, large movements (jumps).

Higher JD Price: The JD model's higher price is a direct measure of the cost of this risk premium. It suggests that a swaption should be valued higher to reflect the actual market volatility, particularly for options far from the money.

Finding: The Black Model systematically underprices the FX Swaption because it neglects the possibility and impact of jumps. The JD Model provides a more prudent (safer) valuation by incorporating this essential jump risk premium, leading to a price that is more reflective of the derivative's true risk profile.

**Theoretical Consistency**

Explaining Tail Risk: The primary finding is related to risk modeling and the volatility smile.

Black Model Flaw: The Black model assumes the underlying asset's price movements follow a log-normal distribution, implying a constant volatility across all strikes. It fails to account for tail risk (the higher-than-normal probability of extreme price movements).

JD Model Superiority: The Merton Jump-Diffusion model explicitly incorporates a Compound Poisson Process for jumps, resulting in a leptokurtic distribution (a fatter tail and sharper peak) than the log-normal distribution.

This "fatter tail" gives a higher probability to the large, extreme moves that lead to the swaption finishing deep In-The-Money (ITM).

Finding: The JD model is theoretically superior because it provides a more robust and realistic representation of the FX market's empirical price distribution, which is known to exhibit heavy tails and skew.